## Mathematical Formulation of Surgery Scheduling Model

This section details the mathematical formulation of the surgery scheduling optimization problem. The model aims to allocate surgical operations to available rooms on different days, minimizing the overall waiting time while adhering to operational constraints.

### Decision Variables

- \( x_{s,d,r} \): Binary variable that equals 1 if surgery \( s \) is scheduled in room \( r \) on day \( d \), and 0 otherwise.

### Objective Function

**Minimize Total Adjusted Waiting Time**:
The objective is to minimize the sum of the products of the waiting times, the priority of the surgeries, and the decision variables across all surgeries, days, and rooms.

$$
\text{Minimize} \quad Z = \sum_{s \in S} \sum_{d \in D} \sum_{r \in R} x_{s,d,r} \cdot ( \text{days\_waited}_s - (d - \text{current\_day})) \cdot \text{priority}_s 
$$

- $ \text{days\_waited}_s $: Days the patient has waited for surgery $ s $, which is the negative value of the admitted days until today.
- $ \text{priority}_s $: Priority level of surgery $ s $, determined by its urgency.
- $ \text{current\_day} $: The day from which scheduling starts.

### Constraints

1. **Single Scheduling Constraint**:
   Each surgery must be scheduled no more than once within the scheduling horizon to ensure there are no double bookings.

   $$
   \sum_{d \in D} \sum_{r \in R} x_{s,d,r} \leq 1 \quad \forall s \in S 

   $$

2. **Capacity Constraints**:
   The total duration of surgeries scheduled in each room on each day must not exceed the dynamically adjusted available capacity. This considers the percentage of capacity already scheduled and adjusts further capacity based on proximity to the current day.

   $$ 
   sum_{s \in S} x_{s,d,r} \cdot \text{duration}_s \leq \text{allowed\_capacity}_{d,r} \quad \forall d \in D, \forall r \in R 
   $$

   Where:
   - $ {allowed\_capacity}_{d,r} $: The minimum of the additional capacity allowed and the current capacity, which is calculated as:

$$
\min\left( \left(\text{NEXT\_DAY} \text{ if } d = \text{current\_day} + 1 \text{ else } \text{LOOKAHEAD}\right) \times \text{original\_capacity}_{d,r} - \text{percentage\_already\_scheduled}_{d,r} \times \text{original\_capacity}_{d,r}, \text{current\_capacity}_{d,r} \right)
$$

- $\text{NEXT\_DAY}$ and $\text{LOOKAHEAD}$ are constants that define how much of the original capacity can be scheduled based on whether the scheduling day is the next day or further in the future.
- $\text{original\_capacity}_{d,r}$ refers to the original total available hours for each room on each day.
- $\text{percentage\_already\_scheduled}_{d,r}$ is the fraction of the original capacity that has already been scheduled for surgeries, calculated as the difference between the original capacity and the current remaining capacity divided by the original capacity.
- $\text{current\_capacity}_{d,r}$ is the remaining available capacity for each room on the day after considering the surgeries already scheduled.

   
 

   

### Model Execution

The model is optimized using the Gurobi Optimizer. Depending on the optimization outcome, different scenarios are handled:
- **Optimal**: If an optimal solution is found, surgeries are scheduled as per the solution.
- **Infeasible**: If the model is infeasible, an IIS is computed to identify conflicting constraints.
- **Unbounded or Other**: Additional statuses are checked and handled accordingly.


## Function: `schedule_surgeries`

This function defines and solves an optimization model for scheduling surgeries over a specified horizon. It prioritizes surgeries based on patient waiting times and the urgency associated with each surgery type.

### Inputs:

- `surgeries_df` (pandas DataFrame): Contains each surgery's details, including a unique identifier (`surgery_id`), the type (`surgery_type`), and the admitted date represented as a negative integer for days waited.
- `surgery_type_attributes` (Dictionary): Stores attributes for each surgery type, such as expected duration (`duration`) and priority (`priority`).
- `current_day` (Integer): The current day number, serving as the starting point for the scheduling period.
- `original_capacities` (pandas DataFrame): The original available hours for each room on each day.
- `current_capacities` (pandas DataFrame): The current available hours for each room, adjusted as surgeries are scheduled.
- `days_ahead` (Integer): The number of days to include in the scheduling horizon.
- `beta` (double): Value between 0-1 which influences how much we prioritize based on priority values.

### Outputs:

- `scheduled_surgeries_df` (pandas DataFrame): Contains the scheduled surgeries with their assigned day and room.
- `current_capacities` (pandas DataFrame): Updated to reflect the remaining capacities after scheduling.
- `unscheduled_surgeries_df` (pandas DataFrame): Contains the surgeries not scheduled in the current iteration.

### Objective Function:

The objective is to minimize the total weighted waiting time for all surgeries, thereby prioritizing urgent cases and those who have been waiting longer.

$$
\text{Minimize} \sum_{s \in S} \sum_{d \in D} \sum_{r \in R} x_{s,d,r} \cdot \beta \cdot (\text{days\_waited}_s + (d - \text{current\_day})) \cdot \text{priority}_s
$$

where:
- $S$ is the set of surgeries.
- $D$ is the set of days in the scheduling horizon.
- $R$ is the set of rooms.
- $x_{s,d,r}$ is a binary decision variable indicating if surgery $s$ is scheduled in room $r$ on day $d$.
- $\text{days\_waited}_s$ is the days waited since admission, with an offset to ensure no zero values.
- $\text{priority}_s$ is the priority level of the surgery.

### Constraints:

**Single Scheduling Constraint:** Each surgery is scheduled at most once within the scheduling horizon.

$$
\sum_{d \in D} \sum_{r \in R} x_{s,d,r} \leq 1 \quad \forall s \in S
$$

**Capacity Constraints:** The total duration of surgeries in each room on each day must not exceed the available capacity, with adjustments for next-day and subsequent-days scheduling.

$$
\sum_{s \in S} x_{s,d,r} \cdot \text{duration}_s \leq \min(\text{allowed\_capacity}_{d,r}, \text{current\_capacity}_{d,r}) \quad \forall d \in D, \forall r \in R
$$

where:
- $\text{allowed\_capacity}_{d,r}$ is either 90% or 20% of the original capacity of room $r$ on day $d$, depending on whether $d$ is the next day or a subsequent days till max look ahead days.
- $\text{current\_capacity}_{d,r}$ is the remaining capacity for room $r$ on day $d$, updated as surgeries are scheduled.

### Execution:

- The model is optimized using the Gurobi optimizer, and the function returns the detailed schedule of surgeries, the updated capacities, and the list of unscheduled surgeries for potential future scheduling.


In [322]:
GLOBAL_NEXT_DAY_CAP = 0.9
GLOBAL_LOOKAHEAD_CAP = 0.2

In [323]:
import pandas as pd
from gurobipy import Model, GRB

def schedule_surgeries(surgeries_df, surgery_type_attributes, current_day, original_capacities, current_capacities, days_ahead=10, beta=0.0):
    # Ensure 'initial_days_waited' is calculated correctly
    if 'initial_days_waited' not in surgeries_df.columns:
        surgeries_df['initial_days_waited'] = -1 * (surgeries_df['admitted_date'])
    
    if surgeries_df.empty:
        print("No surgeries left to schedule.")
        return None, current_capacities, surgeries_df  # Returning the unchanged capacities and an empty DataFrame
    
    if 'surgery_id' not in surgeries_df.columns:
        print("surgery_id column is missing from surgeries_df")

    print(f'day: {current_day}')

    # Initialize the model
    m = Model("SurgeryScheduling")

    # Determine the range of days to schedule
    days = range(current_day + 1, current_day + days_ahead)
    rooms = original_capacities['room'].unique().tolist()

    # Add decision variables for scheduling surgeries
    surgery_scheduled = m.addVars(surgeries_df['surgery_id'].tolist(), days, rooms, vtype=GRB.BINARY, name="schedule")
    # Set the objective function to minimize the total waiting time including scheduling delay
    m.setObjective(sum(surgery_scheduled[s, d, r] * (surgeries_df.at[s, 'initial_days_waited'] - (d - current_day)) * surgery_type_attributes[surgeries_df.at[s, 'surgery_type']]['priority']
                       for s in surgeries_df.index for d in days for r in rooms), GRB.MINIMIZE)

    # Constraints to ensure each surgery is only scheduled once
    for s in surgeries_df.index:
        m.addConstr(sum(surgery_scheduled[s, d, r] for d in days for r in rooms) <= 1, name=f"OneTime_{s}")

    # Dynamic capacity constraints
    for d in days:
        for r in rooms:
            original_capacity = original_capacities.loc[(original_capacities['day'] == d) & (original_capacities['room'] == r), 'capacity'].values[0]
            current_capacity = current_capacities.loc[(current_capacities['day'] == d) & (current_capacities['room'] == r), 'capacity'].values[0]
            percentage_already_scheduled = (original_capacity - current_capacity) / original_capacity
            
            additional_capacity_allowed = (GLOBAL_NEXT_DAY_CAP if (d == current_day + 1) else GLOBAL_LOOKAHEAD_CAP) * original_capacity - percentage_already_scheduled * original_capacity
            allowed_capacity = min(additional_capacity_allowed, current_capacity)

            m.addConstr(sum(surgery_scheduled[s, d, r] * surgery_type_attributes[surgeries_df.at[s, 'surgery_type']]['duration']
                            for s in surgeries_df.index) <= allowed_capacity,
                        name=f"Capacity_{d}_{r}")
            # Print the details of the constraints
            # print(f"Added constraint for day {d}, room {r}:")
            # print(f"  Original Capacity: {original_capacity}")
            # print(f"  Current Capacity: {current_capacity}")
            # print(f"  Percentage Already Scheduled: {percentage_already_scheduled * 100:.2f}%")
            # print(f"  Additional Capacity Allowed: {additional_capacity_allowed}")
            # print(f"  Allowed Capacity (Constraint RHS): {allowed_capacity}\n")

    # Optimize the model
    m.optimize()

    if m.status == GRB.OPTIMAL:
        # Create a list of dictionaries for each scheduled surgery
        scheduled_surgeries = []
        for s in surgeries_df['surgery_id']:  # Loop over the surgery_id column
            for d in days:
                for r in rooms:
                    if surgery_scheduled[s, d, r].X > 0.5:  # Access the variable using the surgery_id
                        # Append a dictionary with the surgery details to the list
                        scheduled_surgeries.append({
                            'surgery_id': s,
                            'day': d,
                            'room': r,
                            'type' : surgeries_df.loc[s, 'surgery_type'],
                            'duration': surgery_type_attributes[surgeries_df.loc[s, 'surgery_type']]['duration']
                        })
                        current_capacities.loc[(current_capacities['day'] == d) & (current_capacities['room'] == r), 'capacity'] -= surgery_type_attributes[surgeries_df.at[s, 'surgery_type']]['duration']
                        current_capacities = current_capacities[[ 'day', 'room', 'capacity']]
        # Creating a DataFrame from the list of dictionaries (scheduled surgeries)
        scheduled_surgeries_df_1 = pd.DataFrame(scheduled_surgeries)
        if scheduled_surgeries_df_1.empty:
            unscheduled_surgeries_df = surgeries_df
        else:
            # Filtering to find unscheduled surgeries
            unscheduled_surgeries_df = surgeries_df[~surgeries_df['surgery_id'].isin(scheduled_surgeries_df_1['surgery_id'])]

        return(scheduled_surgeries_df_1, current_capacities, unscheduled_surgeries_df)

    elif m.status == GRB.INFEASIBLE:
        print("Model is infeasible; computing IIS")
        m.computeIIS()
        print("\nThe following constraint(s) cannot be satisfied:")
        for c in m.getConstrs():
            if c.IISConstr:
                print(f"{c.constrName} is infeasible")

    elif m.status == GRB.UNBOUNDED:
        print('Optimization was unbounded.')

    else:
        print(f'Optimization ended with status {m.status}')


In [324]:
surgery_type_attributes = {
    "Cardiovascular": {"duration": 1, "variance": 1.0, "priority": 3},
    "Neurological": {"duration": 1, "variance": 1.2, "priority": 1},
    "Orthopedic": {"duration": 1, "variance": 0.5, "priority": 1},
    "Gastrointestinal": {"duration": 1, "variance": 0.7, "priority": 1},
    "Cosmetic": {"duration": 1, "variance": 0.3, "priority": 1},
}

surgery_portfolio_df = pd.read_csv("surgery_portfolio.csv")
unscheduled_surgeries_df = surgery_portfolio_df
original_capacities = pd.read_csv('OR_caps.csv')
all_scheduled_surgeries = pd.DataFrame()
current_day = 0  # Starting day for scheduling
days_ahead = 10  # How far ahead to schedule
current_capacities = original_capacities.copy()


In [340]:

scheduled_surgeries_df, updated_capacities, unscheduled_surgeries_df = schedule_surgeries(
    surgeries_df=unscheduled_surgeries_df,
    surgery_type_attributes=surgery_type_attributes,
    current_day=current_day,
    original_capacities=original_capacities,
    current_capacities=current_capacities,
    days_ahead=days_ahead
)

# Update for next iteration
current_capacities = updated_capacities
current_day += 1  # Increment the day

# Collect all scheduled surgeries
all_scheduled_surgeries = pd.concat([all_scheduled_surgeries, scheduled_surgeries_df])

with pd.ExcelWriter(f'Schedule/scheduling_results_day_{current_day}.xlsx') as writer:
    all_scheduled_surgeries.to_excel(writer, sheet_name='All Scheduled Surgeries', index=False)
    updated_capacities.to_excel(writer, sheet_name='Final Updated Capacities', index=False)
    unscheduled_surgeries_df.to_excel(writer, sheet_name='Final Unscheduled Surgeries', index=False)

print(all_scheduled_surgeries.shape)

No surgeries left to schedule.
(250, 5)


In [326]:
scheduled_surgeries_df


,surgery_id,day,room,type,duration
0,0,1,0,Cardiovascular,1
1,1,1,0,Cardiovascular,1
2,2,1,0,Cardiovascular,1
3,3,1,0,Cardiovascular,1
4,4,1,0,Cardiovascular,1
5,5,1,0,Cardiovascular,1
6,6,1,0,Cardiovascular,1
7,7,1,1,Cardiovascular,1
8,8,1,1,Cardiovascular,1
9,9,1,1,Cardiovascular,1


In [327]:
all_scheduled_surgeries

,surgery_id,day,room,type,duration
0,0,1,0,Cardiovascular,1
1,1,1,0,Cardiovascular,1
2,2,1,0,Cardiovascular,1
3,3,1,0,Cardiovascular,1
4,4,1,0,Cardiovascular,1
5,5,1,0,Cardiovascular,1
6,6,1,0,Cardiovascular,1
7,7,1,1,Cardiovascular,1
8,8,1,1,Cardiovascular,1
9,9,1,1,Cardiovascular,1


In [328]:
updated_capacities.head(30)

,day,room,capacity
0,1,0,1
1,1,1,1
2,1,2,1
3,2,0,7
4,2,1,7
5,2,2,7
6,3,0,7
7,3,1,7
8,3,2,7
9,4,0,7


In [329]:
unscheduled_surgeries_df

,surgery_id,surgery_type,admitted_date,initial_days_waited
45,45,Cardiovascular,1,-1
46,46,Cardiovascular,1,-1
47,47,Cardiovascular,1,-1
48,48,Cardiovascular,1,-1
49,49,Cardiovascular,1,-1
...,...,...,...,...
245,245,Cosmetic,1,-1
246,246,Cosmetic,1,-1
247,247,Cosmetic,1,-1
248,248,Cosmetic,1,-1
